In [24]:
import pandas as pd
import numpy as np
import math

In [1]:
# cash flow simulations

In [ ]:
https://www.occ.treas.gov/publications-and-resources/publications/comptrollers-handbook/files/asset-securitization/pub-ch-asset-securitization.pdf
    page 54

In [28]:
# sim 1 

loan_start_date = '09-01-2020'
loan_maturity_date = '08-31-2050'
# (pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) // np.timedelta64(1, 'M')
# math.ceil((pd.to_datetime(loan_maturity_date) - pd.to_datetime(loan_start_date)) / np.timedelta64(1, 'M'))
number_of_pds = 360

loan_amnt = 500000
annual_ir = .045
monthly_ir = annual_ir / 12

In [39]:
# create dates

dates_index = pd.date_range(loan_start_date, loan_maturity_date, freq = 'M')

In [41]:
df = pd.DataFrame(columns = ['Period', 'Payment', 'Interest', 'Principal', 'Residual Balance'])
df

,Period,Payment,Interest,Principal,Residual Balance


In [42]:
df['Period'] = dates_index
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,NaN,NaN,NaN,NaN
1,2020-10-31,NaN,NaN,NaN,NaN
2,2020-11-30,NaN,NaN,NaN,NaN
3,2020-12-31,NaN,NaN,NaN,NaN
4,2021-01-31,NaN,NaN,NaN,NaN
...,...,...,...,...,...
355,2050-04-30,NaN,NaN,NaN,NaN
356,2050-05-31,NaN,NaN,NaN,NaN
357,2050-06-30,NaN,NaN,NaN,NaN
358,2050-07-31,NaN,NaN,NaN,NaN


In [47]:
# monthly payment formula
# M = P [ i(1 + i)^n ] / [ (1 + i)^n – 1]

monthly_payment = loan_amnt * ((monthly_ir * (monthly_ir + 1) ** number_of_pds) / (((monthly_ir + 1) ** (number_of_pds)) - 1))
monthly_payment = round(monthly_payment, 2)
monthly_payment

2533.43

In [48]:
df['Payment'] = monthly_payment

In [49]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,NaN,NaN,NaN
1,2020-10-31,2533.43,NaN,NaN,NaN
2,2020-11-30,2533.43,NaN,NaN,NaN
3,2020-12-31,2533.43,NaN,NaN,NaN
4,2021-01-31,2533.43,NaN,NaN,NaN
...,...,...,...,...,...
355,2050-04-30,2533.43,NaN,NaN,NaN
356,2050-05-31,2533.43,NaN,NaN,NaN
357,2050-06-30,2533.43,NaN,NaN,NaN
358,2050-07-31,2533.43,NaN,NaN,NaN


In [50]:
loan_amnt * monthly_ir

1875.0

In [52]:
round(monthly_payment - (loan_amnt * monthly_ir), 2)

658.43

In [53]:
loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)

499341.57

In [58]:
initial_payment = [loan_amnt * monthly_ir, round(monthly_payment - (loan_amnt * monthly_ir), 2), loan_amnt - round(monthly_payment - (loan_amnt * monthly_ir), 2)]

In [59]:
initial_payment

[1875.0, 658.43, 499341.57]

In [60]:
df.loc[0, ['Interest', 'Principal', 'Residual Balance']]

Interest            NaN
Principal           NaN
Residual Balance    NaN
Name: 0, dtype: object

In [61]:
df.loc[0, ['Interest', 'Principal', 'Residual Balance']] = initial_payment

In [62]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,1875.0,658.43,499341.57
1,2020-10-31,2533.43,NaN,NaN,NaN
2,2020-11-30,2533.43,NaN,NaN,NaN
3,2020-12-31,2533.43,NaN,NaN,NaN
4,2021-01-31,2533.43,NaN,NaN,NaN
...,...,...,...,...,...
355,2050-04-30,2533.43,NaN,NaN,NaN
356,2050-05-31,2533.43,NaN,NaN,NaN
357,2050-06-30,2533.43,NaN,NaN,NaN
358,2050-07-31,2533.43,NaN,NaN,NaN


In [64]:
df.loc[1, ]

Period              2020-10-31 00:00:00
Payment                         2533.43
Interest                            NaN
Principal                           NaN
Residual Balance                    NaN
Name: 1, dtype: object

In [99]:
int_vals = []
prin_vals = []
bal_vals = []

for i in df.index[1:, ]:
    interest = round(monthly_ir * df.iloc[i-1]['Residual Balance'], 2)
    principal = round(df.iloc[i]['Payment'] - interest, 2)
    res_bal = round(df.iloc[i-1]['Residual Balance'] - principal, 2)
    df.loc[i, ['Interest', 'Principal', 'Residual Balance']] = [interest, principal, res_bal]
    int_vals.append(interest)
    prin_vals.append(principal)
    bal_vals.append(res_bal)

In [101]:
# int_vals
# prin_vals
# bal_vals

In [102]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,1875.0,658.43,499341.57
1,2020-10-31,2533.43,1872.53,660.9,498680.67
2,2020-11-30,2533.43,1870.05,663.38,498017.29
3,2020-12-31,2533.43,1867.56,665.87,497351.42
4,2021-01-31,2533.43,1865.07,668.36,496683.06
...,...,...,...,...,...
355,2050-04-30,2533.43,46.96,2486.47,10036.95
356,2050-05-31,2533.43,37.64,2495.79,7541.16
357,2050-06-30,2533.43,28.28,2505.15,5036.01
358,2050-07-31,2533.43,18.89,2514.54,2521.47


In [113]:
# df['Residual Balance'] = df['Residual Balance'].clip(lower = 0)

In [114]:
df

,Period,Payment,Interest,Principal,Residual Balance
0,2020-09-30,2533.43,1875.0,658.43,499341.57
1,2020-10-31,2533.43,1872.53,660.9,498680.67
2,2020-11-30,2533.43,1870.05,663.38,498017.29
3,2020-12-31,2533.43,1867.56,665.87,497351.42
4,2021-01-31,2533.43,1865.07,668.36,496683.06
...,...,...,...,...,...
355,2050-04-30,2533.43,46.96,2486.47,10036.95
356,2050-05-31,2533.43,37.64,2495.79,7541.16
357,2050-06-30,2533.43,28.28,2505.15,5036.01
358,2050-07-31,2533.43,18.89,2514.54,2521.47


In [115]:
display(df['Payment'].sum())
display(df['Interest'].sum())
display(df['Principal'].sum())

912034.8

412032.29999999993

500002.50000000006

In [116]:
df['Residual Balance'].clip(lower = 0)

0      499341.57
1      498680.67
2      498017.29
3      497351.42
4      496683.06
         ...    
355     10036.95
356      7541.16
357      5036.01
358      2521.47
359            0
Name: Residual Balance, Length: 360, dtype: object